# Stochastic differential equation
## Study of differential equation 
## Author: Szymon Pawlowski

## Analytical solution 

We consider a stochastic differential equation of the form It${o}$.
$$
dX(t) = a(t, X(t)) dt + b(t, X(t))dB(t)
$$

The initial form of the considered linear transformation of the SDE is of the form

$$
dZ(t) = (1 - \frac{1}{2}Z(t))dt  - Z(t)dB(t)
$$

with initial condition $Z(0) = 1$.

So we consider the functions

$$
a(t, x) = 1 - \frac{1}{2}x,
$$
$$
b(t, x) = -1
$$.

The analytical solution is of the form

$$
Z(t) = \exp(-t - B(t))\cdot(1 + \int_{0}^{t}{\exp(s + B(s))ds}).
$$

## Euler - Maruyama Method

We assume that there is a stochastic differential equation It$\hat{o}$ 

$$
dX(t) = a(t, X(t)) dt + b(t, X(t))dB(t).
$$

satisfies the linear growth condition, the Lipschitz condition, and the

$$
\forall_{t_1, t_2 \in [0, T]} \: |a(t_1, x) - a(t_2, x)| + (b(t_1, x) - b(t_2, x)| \leq K|t_1 - t_2|^{\frac{1}{2}}.
$$


The SDE we considered as shown in the report satisfies these conditions therefore the Euler-Maruyama method converges and approximates the trajectory $X(t)$ at fixed points $t_i$, $X(t_i) \approx X_i$ on the interval $[0, T]$ based on the formula

$$
X_{i+1} = X_i + a(t_i, X_i)dt + b(t_i, X_i)dB_i
$$

for $i\in \{0, ..., N-1\}$ where

1. $0 = t_0 < t_1 < ... < t_{k-1} < t_k = T$;
2. $dt = t_{i+1} - t_i = T/N$;
3. $dB_i = B(t_{i+1}) - B(t_i) = \sqrt{dt}\eta_i$;
4. $\eta_i \sim \mathcal{N}(0,1)$ i.i.d.;
5. $X_0 = X(0)$;

## Milstein Method

With designations as in the Euler-Maruyama method, we have

$$
X_{i+1} = X_i + a(t_i, X_i)dt + b(t_i, X_i)dB_i + 
\frac{1}{2}b(t_i, X_i)\frac{\partial b}{\partial x}(t_i, X_i)((dB_i)^2 - dt)
$$

## Libraries

In [16]:
import numpy as np
import pandas as pd
import scipy.integrate as integrate
import scipy.special as special
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

## Functions

In [17]:
def a(t, x):
    """
    Definition of function a(t, X_t)
    """
    
    return (-1/2 * x + 1)

def b(t, x):
    """
    Definition of function b(t, X_t)
    """
    
    return (-x)

def db_dx(t,x):
    """
    Definition of function db/dx
    """
    
    return (-1)

In [18]:
def generate_noise(N, seed):
    """
    Function generating random noise on specified seed
    """
    np.random.seed(seed)
    
    return np.random.normal(0, 1, N)

In [19]:
def euler_maruyama(x_0, T, N, seed=123, t_c=0):
    """
    Function returning approximated solution of stochastic differential equation using Euler - Maruyama method
    
    Args:
        x_0 - starting point
        T - defined interval maximum point
        N - number of iterations
        seed - specified seed
        t_c - specified time point
    """
    
    #Initial values
    dt = T/N
    t = np.arange(0, T+dt, dt)    
    X = [x_0]
    
    #Random vector with noise and Brown process shifts
    etas = generate_noise(N, seed)
    dB = np.sqrt(dt)*etas
    #Calculations for the whole time interval
    if t_c == 0:
        for i, t_i in enumerate(t[1:]):
            dB_i = dB[i]
            X_i = X[i]
            X.append(X_i + a(t_i, X_i)*dt + b(t_i, X_i)*dB_i)
    #Calculations for specified time point
    else:
        t = t[t<=t_c]
        for i, t_i in enumerate(t[1:]):
            dB_i = dB[i]
            X_i = X[i]
            X.append(X_i + a(t_i, X_i)*dt + b(t_i, X_i)*dB_i)
        
        
        X = [X[-1].copy()]
        t = [t[-1].copy()]

            
    #Storing results
    dt_X = pd.DataFrame()
    dt_X["t"] = t
    dt_X["X"] = X
        
    return dt_X

In [20]:
def milstein(x_0, T, N, seed=123, t_c=0):
    """
    Function returning approximated solution of stochastic differential equation using Milstein method
    
    Args:
        x_0 - starting point
        T - defined interval maximum point
        N - number of iterations
        seed - specified seed
        t_c - specified time point
    """
    
    #Initial values
    dt = T/N
    t = np.arange(0, T+dt, dt)    
    X = [x_0]

    #Random vector with noise and Brown process shifts
    etas = generate_noise(N, seed)
    dB = np.sqrt(dt) * etas
    #Calculations for the whole time interval
    if t_c == 0:
        for i, t_i in enumerate(t[1:]):
            dB_i = dB[i]
            X_i = X[i]
            X.append(X_i + a(t_i, X_i)*dt + b(t_i, X_i)*dB_i + 1/2 * b(t_i, X_i)*db_dx(t_i, X_i)*(dB_i**2 - dt))
    #Calculations for specified time point
    else:
        t = t[t<=t_c]
        for i, t_i in enumerate(t[1:]):
            dB_i = dB[i]
            X_i = X[i]
            X.append(X_i + a(t_i, X_i)*dt + b(t_i, X_i)*dB_i + 1/2 * b(t_i, X_i)*db_dx(t_i, X_i)*(dB_i**2 - dt))
            
        X = [X[-1].copy()]
        t = [t[-1].copy()]
        
    #Storing results
    dt_X = pd.DataFrame()
    dt_X["t"] = t
    dt_X["X"] = X
        
    return dt_X    

In [21]:
def analytic(x_0, T, N, seed=123, t_c=0):
    """
    Function returning analytical solution of stochastic differential equation
    
    Args:
        x_0 - starting point
        T - defined interval maximum point
        N - number of iterations
        seed - specified seed
        t_c - specified time point
    """
    #Initial values
    dt = T/N
    t = np.arange(0, T+dt, dt)    
    X = [x_0]
    
    
    #Random vector with noise and Brown process shifts
    etas = generate_noise(N, seed)
    brown = np.cumsum(np.sqrt(dt) * etas)
    def brown_int(s, t, brown):
        """
        Function returning value from the stochastic Brown process at the moment s for integral calculations
        """
        t_tmp = t[t <= s][-1]
        pos = np.where(t == t_tmp)[0][0]
        return brown[pos]
        
    #Vector of Brownian motion shifts
    dB = np.sqrt(dt) * etas
    #Calculations for the whole time interval
    if t_c == 0:
        for i, t_i in enumerate(t[1:]):
            B = brown[i] 
            integral = integrate.quad(lambda s: np.exp(s + brown_int(s, t=t, brown=brown)), 0, t_i)[0]
            X.append((np.exp(-t_i - B)) * (x_0 + integral))
    #Calculations for specified time point
    else:
        t=t[t<=t_c]
        for i, t_i in enumerate(t[1:]):
            B = brown[i] 
            integral = integrate.quad(lambda s: np.exp(s + brown_int(s, t=t, brown=brown)), 0, t_i)[0]
            X = [(np.exp(-t_i - B)) * (x_0 + integral)]
            
        t = [t[-1]]    
        X = [X[-1]]
    
    #Storing results
    dt_X = pd.DataFrame()
    dt_X["t"] = t
    dt_X["X"] = X
        
    return dt_X    

In [22]:
def calculate_moments(m, L, method, N):
    """
    Function calculates specified moments
    
    Args:
        m - specified moment
        L - number of simulations
        method - method selected
        N - number of iterations
    """
    
    if method != 'A':
        #For (E-M)
        if method == 'E-M':
            df_means = pd.concat([euler_maruyama(x_0, T, N, seed=10+i) for i in range(L)], ignore_index=True)
        #For (M)
        elif method == 'M':
            df_means = pd.concat([milstein(x_0, T, N, seed=10+i) for i in range(L)], ignore_index=True)
        #Moments calculation
        if 'm' in m:
            #second moment
            if '2' in m:
                df_means['X'] = df_means['X']**2
            #third moment
            elif '3' in m:
                df_means['X'] = df_means['X']**3
            
            df_means = df_means.groupby(by='t', as_index=False).mean()     
        #variance
        elif m == 'var':
            df_means = df_means.groupby(by='t', as_index=False).var()
        else:
            raise Exception("Not implemented!")
    #Analytical solution
    elif method == 'A':
        dt = T/N
        timestamp = np.arange(0, T+dt, dt)  
        #first moment
        if m == 'm1':
            val = [-np.exp(-1/2 * t) + 2 for t in timestamp]
        #second moment
        elif m == 'm2':
            val = [4*(t + np.exp(-1/2 * t)) - 3 for t in timestamp] 
        #third moment
        elif m == 'm3':
            val = [19/3 * np.exp(3/2 * t) - 2/3*(12*t - 1) - 6*np.exp(-1/2 * t) for t in timestamp]
        elif m == 'var':
            val = [4*t - np.exp(-t) + 8*np.exp(-1/2 * t) - 7 for t in timestamp] 
        else:
            raise Exception("Not implemented!")
        df_means = pd.DataFrame()
        df_means['t'] = timestamp
        df_means['X'] = val
        
    return (df_means)

In [23]:
def plot_moments(df, title):
    """
    Function for plotting calculated moments of SDE with their approximations
    
    Args:
        df - dataframe with results
        title - title of the plots
    """
    # Initialize figure with subplots
    fig = make_subplots(
        rows=2, cols=2, subplot_titles=("N=2^5 and L=100", "N=2^5 and L=1000", "N=2^7 and L=100", "N=2^7 and L=1000")
    )

    #Specify grid for methods and their colors
    methods = ['Analytical', 'Euler-Maruyama','Milstein']
    color_grid = {'Analytical':'blue', 'Euler-Maruyama':'green','Milstein':'red'}
    #Select subsets of data
    plt_11 = df[(df.row == 1) & (df.col == 1)]
    plt_12 = df[(df.row == 1) & (df.col == 2)]
    plt_21 = df[(df.row == 2) & (df.col == 1)]
    plt_22 = df[(df.row == 2) & (df.col == 2)]
    #Add traces
    for m in methods:
        t_11 = plt_11[plt_11.Method == m]
        t_12 = plt_12[plt_12.Method == m]
        t_21 = plt_21[plt_21.Method == m]
        t_22 = plt_22[plt_22.Method == m]
        
        fig.add_trace(go.Scatter(x=t_11.t, y=t_11.X, name=m, line_color=color_grid[m]), row=1, col=1)
        fig.add_trace(go.Scatter(x=t_12.t, y=t_12.X, name=m, line_color=color_grid[m], showlegend=False), row=1, col=2)
        fig.add_trace(go.Scatter(x=t_21.t, y=t_21.X, name=m, line_color=color_grid[m], showlegend=False), row=2, col=1)
        fig.add_trace(go.Scatter(x=t_22.t, y=t_22.X, name=m, line_color=color_grid[m], showlegend=False), row=2, col=2)
    #Update axes
    for i in range(2):
        for j in range(2):
            fig.update_xaxes(title_text='t', row=i+1, col=j+1)
            fig.update_yaxes(title_text=title, row=i+1, col=j+1)

    #Update title and height
    fig.update_layout(title_text="Values "+title, height=700)

    fig.show()

In [24]:
def hist_df(L, method, T, t_c):
    """
    Function returning preprocessed dataframe for histogram plotting
    
    Args:
        L - number of simulations
        method - selected method
    """
    #Analytical solution
    if method == 'A':
        hist_df = pd.concat([analytic(x_0, T, N, seed=10+i, t_c=t_c) for i in range(L)], ignore_index=True)
    #Milstein method
    elif method == 'M':
        hist_df = pd.concat([milstein(x_0, T, N, seed=10+i, t_c=t_c) for i in range(L)], ignore_index=True)
    #Euler-Maruyama method
    elif method == 'E-M':
        hist_df = pd.concat([euler_maruyama(x_0, T, N, seed=10+i, t_c=t_c) for i in range(L)], ignore_index=True)
    else:
        raise Exception("Not implemented")
        
    return (hist_df)

In [25]:
def calc_error_df(L, method, T, N, t_c):
    """
    Function returning mean approximation error
    
    Args:
        L - number of simulations
        method - selected method
        T - defined interval maximum point
        N - number of iterations
        t_c - specified time point
    """
    
    if method == 'M':
        method_df = pd.concat([milstein(x_0, T, N, seed=10+i, t_c = t_c) for i in range(L)], ignore_index=True)
    elif method == 'E-M':
        method_df = pd.concat([euler_maruyama(x_0, T, N, seed=10+i, t_c = t_c) for i in range(L)], ignore_index=True)
    else:
        raise Exception("Not implemented")
    
    analytic_df = pd.concat([analytic(x_0, T, N, seed=10+i, t_c = t_c) for i in range(L)], ignore_index=True)

    error_df = np.abs(analytic_df.X - method_df.X)
    res = error_df.mean()
    
    return (res)

## 1. Presentation of sample process trajectories

In [26]:
#Initial values
x_0 = 1
N = 2**7
T = 10

#Seeds (events) for process realization
seeds = [i for i in np.random.randint(0,100, 5)]

#Storing results
df_analytic = pd.concat([analytic(x_0, T, N, seed=10+i) for i in seeds],
          ignore_index=True)
df_analytic['seed'] = sum([[i]*(N+1) for i in seeds], [])

#Plotting
fig = px.line(df_analytic, x="t", y="X", color="seed",
             labels={"t": "t",
                    "X":"X(t)"},
             title="Sample process trajectories X")
fig.update_layout(showlegend=False)
fig.show()

In the graph above, we can see the stochastic nature of the process and how diverse values it can take on. Certainly, it would be incredibly difficult to find a suitable model of a deterministic nature to appropriately approximate it. Moreover, even based on historical data from a generated sample of this process using econometric or statistical methods, approximation would be a non-trivial task.

## 2. Comparison of realizations for a fixed $³$ stochastic process from the analytical form and the corresponding approximations (EM), (M)

We will first select one particular realization of a stochastic process (some fixed seed) and, based on it, generate the values of this process along with their approximations obtained by the Euler-Maruyama (E-M) and Milstein (M) methods.

In [27]:
#Initial values
x_0 = 1
N = 2**7
T = 10

#Storing results
df_compare = pd.concat([analytic(x_0, T, N, seed=10),
                        milstein(x_0, T, N, seed=10),
                        euler_maruyama(x_0, T, N, seed=10)], ignore_index=True)
df_compare['Method'] = sum([['Analytical'] * (N+1),
                                 ['Euler-Maruyama'] * (N+1),
                                 ['Milstein'] * (N+1)], [])

In [28]:
#Plotting
fig = go.Figure()
#Analytical solution
A = df_compare[df_compare.Method == 'Analytical']
#Euler-Maruyama
EM = df_compare[df_compare.Method == 'Euler-Maruyama']
#Milstein
M = df_compare[df_compare.Method == 'Milstein']


#Add traces
fig.add_trace(go.Scatter(x=A.t, y=A.X,
                    mode='lines',
                    line = dict(width=4,),
                    name='Analytical Solution'))
fig.add_trace(go.Scatter(x=EM.t, y=EM.X,
                    mode='lines',
                    name='Euler-Maruyama Method'))
fig.add_trace(go.Scatter(x=M.t, y=M.X,
                    mode='lines',
                    name='Milstein Method'))
fig.update_layout(title='Comparison of the solution of the process X(t) and its approximation',
                   xaxis_title='t',
                   yaxis_title='X(t)')

fig.show()

We can see how, for the selected realization of the process, its approximation is close to the value determined analytically. As mentioned, doing this by other methods would be a difficult task to say the least, so we can see here the enormous potential of stochastic modeling and the use of stochastic differential equations.

## 3. Examine the function of consecutive moments $\mathbb{E}X(t)$, $\mathbb{E}X^2(t)$, $Var X(t)$, $\mathbb{E}X^3(t)$

We will compare the values of the moments of the considered process for different lengths of intervals $N \in \{2^5, 2^7\}$ and for different number of simulations $L \in \{100, 1000\}$$ along with their approximation.

### 3.1. $E(X(t))$

In [29]:
#Initial values
title='E(X(t))'
N_s = [2**5, 2**7]
L_s = [100, 1000]
T = 5
moment = 'm1'

df_plot = pd.DataFrame()
#For each number of iterations
for i,n in enumerate(N_s):
    #For each number of simulations
    for j,l in enumerate(L_s):
        #Store reults
        df_compare = pd.concat([calculate_moments(m=moment,L=l,N=n,method='A'),
                        calculate_moments(m=moment,L=l,N=n,method='E-M'),
                        calculate_moments(m=moment,L=l,N=n,method='M')], ignore_index=True)
        df_compare['Method'] = sum([['Analytical'] * (n+1),
                                 ['Euler-Maruyama'] * (n+1),
                                 ['Milstein'] * (n+1)], [])
        df_compare['row'] = i+1
        df_compare['col'] = j+1
        df_plot = pd.concat([df_plot, df_compare], ignore_index=True)

#Plot
plot_moments(df_plot, title)

While the values of the process itself approximate satisfactorily well, the values of the features of the process would already require a better method. The first moment, that is, the average value in the approximations has large fluctuations with respect to the analytical solution. However, you can see how changing the size of the division and the number of simulations improves the situation. For this realization, the higher number of iterations introduces larger fluctuations, while the increased number of simulations certainly smooths our approximation.

### 3.2. $E(X^2(t))$

In [30]:
#Initial values
title='E(X^2(t))'
N_s = [2**5, 2**7]
L_s = [100, 1000]
T = 5
df_plot = pd.DataFrame()
moment = 'm2'
#For each number of iterations
for i,n in enumerate(N_s):
    #For each number of simulations
    for j,l in enumerate(L_s):
        #Store reults
        df_compare = pd.concat([calculate_moments(m=moment,L=l,N=n,method='A'),
                        calculate_moments(m=moment,L=l,N=n,method='E-M'),
                        calculate_moments(m=moment,L=l,N=n,method='M')], ignore_index=True)
        df_compare['Method'] = sum([['Analytical'] * (n+1),
                                 ['Euler-Maruyama'] * (n+1),
                                 ['Milstein'] * (n+1)], [])
        df_compare['row'] = i+1
        df_compare['col'] = j+1
        df_plot = pd.concat([df_plot, df_compare], ignore_index=True)
        
#Plot
plot_moments(df_plot, title)

The approximation of the value of the second moment already looks a little worse. At the beginning of the interval with small values, the results are relatively good while with time they start to deviate significantly. Again, increasing the number of simulations improves the situation.

### 3.3. $E(X^3(t))$

In [31]:
#Initial values
title='E(X^3(t))'
N_s = [2**5, 2**7]
L_s = [100, 1000]
T = 5
df_plot = pd.DataFrame()
moment = 'm3'

#For each number of iterations
for i,n in enumerate(N_s):
    #For each number of simulations
    for j,l in enumerate(L_s):
        #Store results
        df_compare = pd.concat([calculate_moments(m=moment,L=l,N=n,method='A'),
                        calculate_moments(m=moment,L=l,N=n,method='E-M'),
                        calculate_moments(m=moment,L=l,N=n,method='M')], ignore_index=True)
        df_compare['Method'] = sum([['Analytical'] * (n+1),
                                 ['Euler-Maruyama'] * (n+1),
                                 ['Milstein'] * (n+1)], [])
        df_compare['row'] = i+1
        df_compare['col'] = j+1
        df_plot = pd.concat([df_plot, df_compare], ignore_index=True)
#Plot
plot_moments(df_plot, title)

The value of the third moment is approximated in a reasonably approximate way only for the beginning of the interval. Later, in the analytical solution evidently the role of the leader is taken over by the exponential function $e^{frac{3}{2}t}$ and causes an increase in the value of the moment. Obviously, this increase exceeds the simple raising of the simulated values to the third power hence such a discrepancy.

### 3.4. $Var X(t)$

In [32]:
#Initial values
title='Var X(t)'
N_s = [2**5, 2**7]
L_s = [100, 1000]
T = 5
df_plot = pd.DataFrame()
moment = 'var'
#For each number of iterations
for i,n in enumerate(N_s):
    #For each number of simulations
    for j,l in enumerate(L_s):
        #Store results
        df_compare = pd.concat([calculate_moments(m=moment,L=l,N=n,method='A'),
                        calculate_moments(m=moment,L=l,N=n,method='E-M'),
                        calculate_moments(m=moment,L=l,N=n,method='M')], ignore_index=True)
        df_compare['Method'] = sum([['Analytical'] * (n+1),
                                 ['Euler-Maruyama'] * (n+1),
                                 ['Milstein'] * (n+1)], [])
        df_compare['row'] = i+1
        df_compare['col'] = j+1
        df_plot = pd.concat([df_plot, df_compare], ignore_index=True)

#Plot
plot_moments(df_plot, title)

The approximations of the variance values are similar in terms of accuracy to the approximations of the second moment function. An increase in the simulation value results in better approximations, but it is still possible to notice a certain discrepancy in the results from the simulation and the analytical solution. 

In summary, the tested methods do an excellent job of approximating the execution of the process itself, but when examining its attributes, an oversimplification relative to analytical methods appears. Nevertheless, the appropriate choice of parameters, including the number of simulations and the length of the interval has the potential to improve the results, but the time to generate them could make it uneconomical.

## 4. Examine the distribution of the process $X(t)$


We will now examine the distribution of the process under consideration based on different time points and the approximation methods adopted. Thus, we will consider a number of moments in time $t \in \{0.5, 1, 5\}$ to study the distribution and compare in doing so the analytical method, (M) and (E-M) for $L=1000$.

### 4.1. $t=0.5$

In [33]:
#Initial values
N = 2**5
x_0 = 1
L=1000
t_c = 0.5

#Store results 
df_tmp = pd.concat([hist_df(L=L,method='A', T=t_c, t_c=t_c),
                        hist_df(L=L,method='M', T=t_c, t_c=t_c),
                        hist_df(L=L,method='E-M', T=t_c, t_c=t_c)], ignore_index=True)
df_tmp['Method'] = sum([['Analytical'] * L,
                        ['Milsteina'] * L,
                        ['Cauchy-Maruyamy'] * L], [])

#Plotting
fig = px.histogram(df_tmp,
                    nbins=70, 
                    x="X",
                   color='Method', 
                   labels={"X":"X(t)"}, 
                   opacity=0.7, 
                   marginal="violin",
                   title='Process distribution X(t) for t='+str(t_c))
fig.show()

For the point $t=0.5$, we can observe a high density at $1$ mainly for approximation methods, where the analytical solution is more "flatly" distributed. As conjectured, these distributions somewhat resemble those of the family of exponential distributions with appropriate parameters.

### 4.2. $t=1$

In [34]:
#Initial values
t_c = 1

#Store reults
df_tmp = pd.concat([hist_df(L=L,method='A', T=t_c, t_c=t_c),
                        hist_df(L=L,method='M', T=t_c, t_c=t_c),
                        hist_df(L=L,method='E-M', T=t_c, t_c=t_c)], ignore_index=True)
df_tmp['Method'] = sum([['Analytical'] * L,
                        ['Milsteina'] * L,
                        ['Cauchy-Maruyamy'] * L], [])

#Plotting
fig = px.histogram(df_tmp,
                    nbins=70, 
                    x="X",
                   color='Method', 
                   labels={"X":"X(t)"}, 
                   opacity=0.7, 
                   marginal="violin",
                   title='Process distribution X(t) for t='+str(t_c))
fig.show()

For $t=1$ the situation changes slightly, the analytical solution begins to be denser at $1$, and the distribution actually begins to resemble more of an exponential distribution. In contrast, the solution obtained by the algorithms behave similarly to the previous point and also oscillate strongly at one point. In addition, they also already point more to an exponential distribution.

### 4.3. $t=5$

In [35]:
#Initial values
t_c = 5

#Store results
df_tmp = pd.concat([hist_df(L=L,method='A', T=t_c, t_c=t_c),
                        hist_df(L=L,method='M', T=t_c, t_c=t_c),
                        hist_df(L=L,method='E-M', T=t_c, t_c=t_c)], ignore_index=True)
df_tmp['Method'] = sum([['Analytical'] * L,
                        ['Milsteina'] * L,
                        ['Cauchy-Maruyamy'] * L], [])

#Plotting
fig = px.histogram(df_tmp,
                    nbins=70, 
                    x="X",
                   color='Method', 
                   labels={"X":"X(t)"}, 
                   opacity=0.7, 
                   marginal="violin",
                   title='Process distribution X(t) for t='+str(t_c))
fig.show()

For $t=5$, the main cluster occurs in the interval $[0,1]$. Moreover, the exponential nature of the distribution of the process becomes much more pronounced. Note also that based on this point, as well as the previous ones, it can be concluded that the process does not take negative values due to exponential functions. Of course, one could try to fit many other distributions with appropriate parameters here, but as mentioned, the analytical solution resembles the distribution of a distribution from the family of exponential distributions.

## Approximation error

For a fixed division $dt$ at time $T$, we define the approximation error as

$$
e_{dt} = \mathbb{E}(|X(T) - X_N|)
$$

Approximation converges strongly of order $\gamma > 0$ if $e_{dt} = O((dt)^\gamma)$.

## 5. Investigate the rate of convergence of the (EM) and (M) methods

We will study the absolute error for $L \in \{100, 200\}$ simulation at the point $T=1$ and the value of the $dt \in \{2^{-5}, 2^{-6}, 2^{-7}, 2^{-8}, 2^{-9}, 2^{-10}\}$. 

We will compare them to the theoretical values. The Euler-Maruyama method is of strong order $\gamma = \frac{1}{2}$ so we will compare it to the simple $y=\frac{1}{2}x + b$, $bin \mathbb{R}$. Milstein's method, on the other hand, is of strong order $gamma = 1$ so we will compare it to the simple $y=x + b$, $b\in \mathbb{R}$. We will show the comparative results in log-log plot and tabular form.

### 5.1. Absolute error for $L=10$

In [36]:
#Initial values
x_0 = 1
T = 1
L = 10
t_c = 1

#Approximation errors calculations
Ns = [2**(5+i) for i in range(6)]
dts = [2**(-5-i) for i in range(6)]
M_edts = [calc_error_df(L, 'M', T, n, t_c = t_c) for n in Ns]
EM_edts = [calc_error_df(L, 'E-M', T, n, t_c = t_c) for n in Ns]

In [37]:
#Create subplots
fig = make_subplots(
        rows=2, cols=1, subplot_titles=("Milstein method", "Euler-Maruyama method")
    )

## Milstein
#Theoretical line
y_m = [1*n + 0 for n in np.log2(dts)]

#Add traces
fig.add_trace(go.Scatter(x=np.log2(dts), y=np.log2(M_edts),
                    mode='markers',
                    name='(M) e_dt'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.log2(dts), y=y_m,
                    mode='lines+markers',
                    name='y=x+b'), row=1, col=1)

fig.update_yaxes(title_text='log2(e_dt)', row=1, col=1)
fig.update_xaxes(title_text='log2(dt)', row=1, col=1)
        
## Euler-Maruyama
#Theoretical line
y_em = [n/2 +0 for n in np.log2(dts)]

#Add traces
fig.add_trace(go.Scatter(x=np.log2(dts), y=np.log2(EM_edts),
                    mode='markers',
                    name='(E-M) e_dt'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.log2(dts), y=y_em,
                    mode='lines+markers',
                    name='y=x/2+b'), row=2, col=1)

fig.update_yaxes(title_text='log2(e_dt)', row=2, col=1)
fig.update_xaxes(title_text='log2(dt)', row=2, col=1)

fig.update_layout(title_text="The values of the average approximation error for the L=10", height=700)

fig.show()

We can observe that both methods deviate slightly from the theoretical values, but they are characterized by the same trend, i.e. the error values increase as the time interval increases. Let's move on to compare the two methods.

In [38]:
#Theoretical line
y = [n +0 for n in np.log2(dts)]

#Plotting comparison
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.log2(dts), y=np.log2(EM_edts),
                    mode='markers',
                    name='Euler-Maruyama e_dt'))
fig.add_trace(go.Scatter(x=np.log2(dts), y=np.log2(M_edts),
                    mode='markers',
                    name='Milstein e_dt'))
fig.add_trace(go.Scatter(x=np.log2(dts), y=y,
                    mode='lines+markers',
                    name='y=x+b'))
fig.update_layout(title='Comparison of average approximation errors X(1)',
                   xaxis_title='dt',
                   yaxis_title='e_dt')

fig.update_yaxes(title_text='log2(e_dt)')
fig.update_xaxes(title_text='log2(dt)')

fig.show()

Comparing the two methods, one can see differences in the value of the approximation error, and it can be assumed that Milstein's method handles the approximation task better, but let's put them together in a table without scaling.

In [39]:
#Store results together for further analysis
edts_f1 = pd.DataFrame()
edts_f1['dt'] = dts
edts_f1['log_dt'] = np.log(dts)
edts_f1['EM'] = EM_edts
edts_f1['log_EM'] = np.log(EM_edts)
edts_f1['M'] = M_edts
edts_f1['log_M'] = np.log(M_edts)
edts_f1

,dt,log_dt,EM,log_EM,M,log_M
0,0.031250,-3.465736,0.091286,-2.393760,0.015418,-4.172225
1,0.015625,-4.158883,0.049530,-3.005168,0.004729,-5.354067
2,0.007812,-4.852030,0.062026,-2.780195,0.008017,-4.826168
3,0.003906,-5.545177,0.165200,-1.800597,0.010001,-4.605119
4,0.001953,-6.238325,0.053041,-2.936682,0.002770,-5.889079
5,0.000977,-6.931472,0.034159,-3.376717,0.000881,-7.034934


| $dt$ | Error $e_{dt}$ Milstein | Error $e_{dt}$ Euler-Maruyama |
| --- | --- | --- |
| $2^{-5}$ | 0.015418 | 0.091286 |
| $2^{-6}$ | 0.004729 | 0.049530 |
| $2^{-7}$ | 0.008017 | 0.062026 |
| $2^{-8}$ | 0.010001 | 0.165200 |
| $2^{-9}$ | 0.002770 | 0.053041 |
| $2^{-10}$ | 0.000881 | 0.034159	 |


Indeed, the error of the Milstein method has lower values in general compared to the error of the Euler-Maruyama method. The value of the error of Milstein's method generally decreases as the value of the pitch decreases, as do the values of the error of the Euler-Maruyama method. It is worth noting that the values of the approximation error of the Milstein method according to the theoretical order of convergence decrease more rapidly with an increase in the value of the pitch (thus reducing the values of the intervals). Let's check the situation for $L=50$.

### 5.2. Absolute error for $L=50$

In [40]:
#Initial values
T = 1
L = 50
t_c = 1

#Approximation errors calculations
Ns_l2 = [2**(5+i) for i in range(6)]
dts_l2 = [2**(-5-i) for i in range(6)]
M_edts_l2 = [calc_error_df(L, 'M', T, n, t_c = t_c) for n in Ns]
EM_edts_l2 = [calc_error_df(L, 'E-M', T, n, t_c = t_c) for n in Ns]

In [41]:
#Create subplots
fig = make_subplots(
        rows=2, cols=1, subplot_titles=("Milstein method", "Euler-Maruyama method")
    )

## Milstein
#Theoretical line
y_m_l2 = [1*n - 0.5 for n in np.log2(dts_l2)]
#Add traces
fig.add_trace(go.Scatter(x=np.log2(dts_l2), y=np.log2(M_edts_l2),
                    mode='markers',
                    name='(M) e_dt'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.log2(dts_l2), y=y_m_l2,
                    mode='lines+markers',
                    name='y=x+b'), row=1, col=1)

fig.update_yaxes(title_text='log2(e_dt)', row=1, col=1)
fig.update_xaxes(title_text='log2(dt)', row=1, col=1)

        
## Euler-Maruyama
#Theoretical line
y_em_l2 = [n/2 -0.5 for n in np.log2(dts_l2)]

#Add traces
fig.add_trace(go.Scatter(x=np.log2(dts_l2), y=np.log2(EM_edts_l2),
                    mode='markers',
                    name='(E-M) e_dt'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.log2(dts_l2), y=y_em_l2,
                    mode='lines+markers',
                    name='y=x/2+b'), row=2, col=1)

fig.update_yaxes(title_text='log2(e_dt)', row=2, col=1)
fig.update_xaxes(title_text='log2(dt)', row=2, col=1)

fig.update_layout(title_text="The values of the average approximation error for the L="+str(L), height=700)

fig.show()

We can see that the increased number of simulations changes the situations in the context of approximation to the theoretical values. Milstein's method is almost completely covered by the simple $y=x+b$, which indicates that the order of convergence of the theoretical to the real is consistent. For the Euler-Maruyama method, there is also an apparent improvement, but with slightly larger deviations. Again, let's compare the methods to each other.

In [42]:
#Theoretical line
y_l2 = [n - 0.5 for n in np.log2(dts_l2)]

#Plotting comparison
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.log2(dts_l2), y=np.log2(EM_edts_l2),
                    mode='markers',
                    name='Euler-Maruyama e_dt'))
fig.add_trace(go.Scatter(x=np.log2(dts_l2), y=np.log2(M_edts_l2),
                    mode='markers',
                    name='Milstein e_dt'))
fig.add_trace(go.Scatter(x=np.log2(dts_l2), y=y_l2,
                    mode='lines+markers',
                    name='y=x+b'))
fig.update_layout(title='Comparison of average approximation errors X(1)',
                   xaxis_title='dt',
                   yaxis_title='e_dt')
fig.update_yaxes(title_text='log2(e_dt)')
fig.update_xaxes(title_text='log2(dt)')

fig.show()

Once again, we are able to observe higher error values for the Euler-Maruyama method than for the Milstein method. We can also see the juxtaposition of different orders of convergence by the faster decrease in the approximation error of Milstein's method. Let's see the results in tabular form.

In [43]:
#Store results for further analysis
edts_f2 = pd.DataFrame()
edts_f2['dt'] = dts_l2
edts_f2['log_dt'] = np.log(dts_l2)
edts_f2['EM'] = EM_edts_l2
edts_f2['log_EM'] = np.log(EM_edts_l2)
edts_f2['M'] = M_edts_l2
edts_f2['log_M'] = np.log(M_edts_l2)
edts_f2

,dt,log_dt,EM,log_EM,M,log_M
0,0.031250,-3.465736,0.100519,-2.297404,0.014918,-4.205176
1,0.015625,-4.158883,0.057749,-2.851641,0.006474,-5.039898
2,0.007812,-4.852030,0.041709,-3.177027,0.004051,-5.508905
3,0.003906,-5.545177,0.052885,-2.939632,0.003145,-5.761829
4,0.001953,-6.238325,0.032787,-3.417729,0.001503,-6.499990
5,0.000977,-6.931472,0.020148,-3.904673,0.000666,-7.314483


| $dt$ | Error $e_{dt}$ Milstein | Error $e_{dt}$ Euler-Maruyama |
| --- | --- | --- |
| $2^{-5}$ | 0.014918	 | 0.100519	 |
| $2^{-6}$ | 0.006474	 | 0.057749	 |
| $2^{-7}$ | 0.004051	 | 0.041709 |
| $2^{-8}$ | 0.003145 | 0.052885 |
| $2^{-9}$ | 0.001503	 | 0.032787 |
| $2^{-10}$ | 0.000666 | 0.020148 |


The table shows confirmation of the superiority of the Milstein method over the Euler-Maruyama method in terms of approximation error. Milstein's method achieves relatively lower values and converges more quickly to zero error as the time differences decrease. Already for the number of $L ƒin ${10, 50}$ of simulations, however, the differences are negligible, so we can assume that with an increased number of simulations the averaged values would be similar and the difference will be mainly influenced by the value of the division. Moreover, the Euler-Maruyama method is simpler to obtain because of the consideration of fewer factors. Nevertheless, the above analysis also suggests that when using Milstein's method, a small number of simulations is sufficient to obtain approximations similar to analytical solutions, which is a huge advantage in terms of the required computational time. Ultimately, the choice of method depends on many factors and it is worth considering both options.